In [1]:
import tensorflow as tf
import numpy as np
import cv2
import os

C:\Users\haho6\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from keras.preprocessing.image import load_img, img_to_array
files = os.listdir(r'.\AnimeFaceDetecter\Datas\result\face')
images = []
for i in files:
    image = load_img(r'.\AnimeFaceDetecter\Datas\result\face_gray\{}'.format(i),True,target_size=(128,128))
    image = img_to_array(image)
#    print(image.shape)
    image = image / 256
    images.append(image.flatten())

Using TensorFlow backend.


In [3]:
print(np.array(images).shape)
a = (np.array(images[7]).reshape(128,128,1))*256 
print(a.shape)
cv2.imshow('qwe',a)
cv2.imwrite('qwe.jpg', a)

(1494, 16384)
(128, 128, 1)


True

In [4]:
cv2.destroyAllWindows()

In [126]:
def windowUp(arr):
    frame = cv2.cvtColor(np.array(arr), cv2.COLOR_BGR2RGB)
    cv2.imshow("test", frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [5]:
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

In [19]:

mb_size = 128
X_dim = 128*128
z_dim = 100
h_dim = 128

X = tf.placeholder(tf.float32, shape=[None, X_dim])

D_W1 = tf.Variable(xavier_init([X_dim, h_dim]))
D_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

D_W2 = tf.Variable(xavier_init([h_dim, 1]))
D_b2 = tf.Variable(tf.zeros(shape=[1]))

theta_D = [D_W1, D_W2, D_b1, D_b2] # D를 학습하면서 고려할 값들


z = tf.placeholder(tf.float32, shape=[None, z_dim])

G_W1 = tf.Variable(xavier_init([z_dim, h_dim]))
G_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

G_W2 = tf.Variable(xavier_init([h_dim, X_dim]))
G_b2 = tf.Variable(tf.zeros(shape=[X_dim]))

theta_G = [G_W1, G_W2, G_b1, G_b2] # G를 학습하면서 고려할 변수들


def sample_z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])


def generator(z):
    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)
    return G_prob


def discriminator(x):
    D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
    out = tf.matmul(D_h1, D_W2) + D_b2
    return out


G_sample = generator(z)
D_real = discriminator(X)
D_fake = discriminator(G_sample)

D_loss = tf.reduce_mean(D_real) - tf.reduce_mean(D_fake)
G_loss = -tf.reduce_mean(D_fake)

D_solver = (tf.train.RMSPropOptimizer(learning_rate=1e-4)
            .minimize(-D_loss, var_list=theta_D))
G_solver = (tf.train.RMSPropOptimizer(learning_rate=1e-4)
            .minimize(G_loss, var_list=theta_G))

sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [20]:
np.array(images[3*mb_size:(3+1)*mb_size]).shape

(128, 16384)

In [23]:
mb_size = 64
for it in range(20):
    for i in range(5):
        X_mb = images[0:mb_size]

        _, D_loss_curr = sess.run(
            [D_solver, D_loss],
            feed_dict={X: X_mb, z: sample_z(mb_size, z_dim)}
        )

    _, G_loss_curr = sess.run(
        [G_solver, G_loss],
        feed_dict={z: sample_z(mb_size, z_dim)}
    )
    print('Iter: {}; D loss: {:.4}; G_loss: {:.4}'
              .format(it, D_loss_curr, G_loss_curr))
    if it % 100 == 0:
        samples = sess.run(G_sample, feed_dict={z: sample_z(16, z_dim)})
        for i in range(5):
            
            print("save - %d"%c)
            cv2.imwrite('{}-{}.jpg'.format(c,i), samples[0].reshape(128,128,1)*256)
        c+=1
    if it % 300 == 0:
        #np.random.shuffle(images)
        pass

Iter: 0; D loss: 0.9524; G_loss: -0.1473
save - 1
save - 1
save - 1
save - 1
save - 1
Iter: 1; D loss: 1.811; G_loss: -0.8252
Iter: 2; D loss: 2.65; G_loss: -1.625
Iter: 3; D loss: 3.654; G_loss: -2.534
Iter: 4; D loss: 4.991; G_loss: -3.405
Iter: 5; D loss: 6.672; G_loss: -4.45
Iter: 6; D loss: 8.694; G_loss: -5.665
Iter: 7; D loss: 11.27; G_loss: -7.383
Iter: 8; D loss: 14.27; G_loss: -9.571
Iter: 9; D loss: 18.2; G_loss: -12.15
Iter: 10; D loss: 22.94; G_loss: -14.93
Iter: 11; D loss: 28.04; G_loss: -18.23
Iter: 12; D loss: 34.63; G_loss: -21.73
Iter: 13; D loss: 41.09; G_loss: -26.21
Iter: 14; D loss: 49.0; G_loss: -31.63
Iter: 15; D loss: 56.2; G_loss: -36.66
Iter: 16; D loss: 65.26; G_loss: -42.71
Iter: 17; D loss: 74.31; G_loss: -48.32
Iter: 18; D loss: 83.94; G_loss: -54.4
Iter: 19; D loss: 94.53; G_loss: -60.89


In [11]:

D_b2 = tf.Variable(tf.zeros(shape=[1]))

In [22]:
c = 1